In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = r'C:\Users\rusha\OneDrive\Desktop\AQI_Prediction_Project\data\air quality data.csv'  # Adjust path if needed
df = pd.read_csv(file_path, parse_dates=['Date'])

In [4]:
# Add Month Column
df['Month'] = df['Date'].dt.month

In [5]:
# Fill Missing Pollutant Data (Group by City & Month)
for col in ['PM2.5', 'PM10', 'NO2', 'NH3', 'SO2', 'CO', 'O3']:
    df[col] = df.groupby(['City', 'Month'])[col].transform(lambda x: x.fillna(x.median()))

C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
 

In [6]:
# Fill AQI (Median AQI per City & Month)
df['AQI'] = df.groupby(['City', 'Month'])['AQI'].transform(lambda x: x.fillna(x.median()))

C:\Users\rusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [7]:
# Recalculate AQI_Bucket (Based on Filled AQI)
def get_aqi_bucket(aqi):
    if aqi <= 50:
        return 'Good'
    elif aqi <= 100:
        return 'Satisfactory'
    elif aqi <= 200:
        return 'Moderate'
    elif aqi <= 300:
        return 'Poor'
    elif aqi <= 400:
        return 'Very Poor'
    else:
        return 'Severe'

df['AQI_Bucket'] = df['AQI'].apply(get_aqi_bucket)

In [8]:
# Add Season
df['Season'] = df['Month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Summer', 4: 'Summer', 5: 'Summer',
    6: 'Monsoon', 7: 'Monsoon', 8: 'Monsoon',
    9: 'Post-Monsoon', 10: 'Post-Monsoon', 11: 'Post-Monsoon'
})

In [9]:
# Add Weekend Flag
df['Weekend'] = df['Date'].dt.weekday >= 5

In [10]:
# Add Pollutant Ratios
df['PM2.5/PM10'] = df['PM2.5'] / df['PM10']
df['NO2/SO2'] = df['NO2'] / df['SO2']

In [11]:
# Add Previous Day AQI (Lag Feature)
df = df.sort_values(by=['City', 'Date'])
df['Previous_Day_AQI'] = df.groupby('City')['AQI'].shift(1)

In [12]:
# Drop any remaining rows with missing values after all processing
df = df.dropna()

In [13]:
# Final Null Value Check
print("Null Values after Processing:\n", df.isnull().sum())

Null Values after Processing:
 City                0
Date                0
PM2.5               0
PM10                0
NO                  0
NO2                 0
NOx                 0
NH3                 0
CO                  0
SO2                 0
O3                  0
Benzene             0
Toluene             0
Xylene              0
AQI                 0
AQI_Bucket          0
Month               0
Season              0
Weekend             0
PM2.5/PM10          0
NO2/SO2             0
Previous_Day_AQI    0
dtype: int64


In [14]:
# Confirm Final Data Shape
print("\nFinal Data Shape:", df.shape)


Final Data Shape: (7848, 22)


In [15]:
# Sample Rows from Cleaned Data
print("\nSample Rows from Cleaned Data:\n", df.head())


Sample Rows from Cleaned Data:
            City       Date  PM2.5    PM10    NO    NO2    NOx    NH3    CO  \
2123  Amaravati 2017-11-25  81.40  124.50  1.44  20.50  12.08  10.72  0.12   
2124  Amaravati 2017-11-26  78.32  129.06  1.26  26.00  14.85  10.28  0.14   
2125  Amaravati 2017-11-27  88.76  135.32  6.60  30.85  21.77  12.91  0.11   
2126  Amaravati 2017-11-28  64.18  104.09  2.56  28.07  17.01  11.42  0.09   
2127  Amaravati 2017-11-29  72.47  114.84  5.23  23.20  16.59  12.25  0.16   

        SO2  ...  Toluene  Xylene    AQI  AQI_Bucket  Month        Season  \
2123  15.24  ...     6.50    0.06  184.0    Moderate     11  Post-Monsoon   
2124  26.96  ...     7.95    0.08  197.0    Moderate     11  Post-Monsoon   
2125  33.59  ...     7.63    0.12  198.0    Moderate     11  Post-Monsoon   
2126  19.00  ...     5.02    0.07  188.0    Moderate     11  Post-Monsoon   
2127  10.55  ...     4.71    0.08  173.0    Moderate     11  Post-Monsoon   

      Weekend PM2.5/PM10   NO2/SO2 

In [16]:
# Save the cleaned file for modeling
df.to_csv('../data/cleaned_air_quality_data.csv', index=False)

In [3]:
# Load Cleaned Data (After Filling Missing, Adding Features)
df = pd.read_csv('../data/cleaned_air_quality_data.csv', parse_dates=['Date'])

# ✅ Outlier Removal - IQR Method (Applied to All Pollutants)
pollutants = ['PM2.5', 'PM10', 'NO2', 'NH3', 'SO2', 'CO', 'O3']

for col in pollutants:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove Outliers
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# ✅ Recalculate Pollutant Ratios (since some values changed after outliers removal)
df['PM2.5/PM10'] = df['PM2.5'] / df['PM10']
df['NO2/SO2'] = df['NO2'] / df['SO2']

# ✅ Recalculate Previous Day AQI (after row drops due to outliers)
df = df.sort_values(by=['City', 'Date'])
df['Previous_Day_AQI'] = df.groupby('City')['AQI'].shift(1)

# ✅ Drop any rows with NaN after recalculations
df = df.dropna()

# ✅ Final Check for Null Values (Safety)
print("\n✅ Final Null Value Check (after outlier removal):")
print(df.isnull().sum())

# ✅ Confirm Final Shape
print(f"\n✅ Final Data Shape (after outlier removal): {df.shape}")

# ✅ Save Final Data (Overwrite if confident, or save as new file)
df.to_csv('../data/cleaned_air_quality_data.csv', index=False)

print("\n✅ Data Saved Successfully to 'cleaned_air_quality_data.csv'")



✅ Final Null Value Check (after outlier removal):
City                0
Date                0
PM2.5               0
PM10                0
NO                  0
NO2                 0
NOx                 0
NH3                 0
CO                  0
SO2                 0
O3                  0
Benzene             0
Toluene             0
Xylene              0
AQI                 0
AQI_Bucket          0
Month               0
Season              0
Weekend             0
PM2.5/PM10          0
NO2/SO2             0
Previous_Day_AQI    0
dtype: int64

✅ Final Data Shape (after outlier removal): (5530, 22)

✅ Data Saved Successfully to 'cleaned_air_quality_data.csv'
